In [1]:
import pandas as pd

## Getting tracts with their respective coordinates

In [2]:
tracts = pd.read_csv('./Mapping/tract_coordinates.csv')
tracts.head()

,tractid,latitude,longitude
0,6075010100,37.808420,-122.408579
1,6075010200,37.805861,-122.422925
2,6075010300,37.801620,-122.415681
3,6075010400,37.802273,-122.407558
4,6075010500,37.802684,-122.399059


## Fetching crime data

In [3]:
data_path = './CensusData/sfo data/'

In [4]:
crimes = pd.concat([pd.read_csv(data_path+'crime_data_{}.csv'.format(k)) for k in xrange(10)])
crimes.head()

,Unnamed: 0,﻿IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,0,150027849,NON-CRIMINAL,SEARCH WARRANT SERVICE,Friday,01/09/2015,22:02,NORTHERN,"ARREST, BOOKED",200 Block of LAGUNA ST,-122.425723,37.773431,"(37.7734305349811, -122.425722704575)",15002784975025
1,1,150046504,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Thursday,01/15/2015,19:00,SOUTHERN,NONE,400 Block of THE EMBARCADEROSOUTH ST,-122.396709,37.797873,"(37.7978728855933, -122.39670853026)",15004650428160
2,2,150048817,LARCENY/THEFT,PETTY THEFT SHOPLIFTING,Friday,01/16/2015,16:26,CENTRAL,NONE,200 Block of GRANT AV,-122.405254,37.789253,"(37.7892525040522, -122.405254463024)",15004881706363
3,3,150380578,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,Friday,05/01/2015,16:44,TARAVAL,"ARREST, BOOKED",1900 Block of 19TH AV,-122.476096,37.751433,"(37.7514328843776, -122.476096065298)",15038057806302
4,4,150380653,NON-CRIMINAL,"LICENSE PLATE, FOUND",Thursday,04/30/2015,23:00,RICHMOND,NONE,BRODERICK ST / CALIFORNIA ST,-122.442112,37.787856,"(37.7878555996754, -122.442112423143)",15038065371011


In [5]:
# Weed out the non-criminal crimes
real_crimes = crimes[crimes['Category']!='NON-CRIMINAL']
real_crimes.head()

,Unnamed: 0,﻿IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
1,1,150046504,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Thursday,01/15/2015,19:00,SOUTHERN,NONE,400 Block of THE EMBARCADEROSOUTH ST,-122.396709,37.797873,"(37.7978728855933, -122.39670853026)",15004650428160
2,2,150048817,LARCENY/THEFT,PETTY THEFT SHOPLIFTING,Friday,01/16/2015,16:26,CENTRAL,NONE,200 Block of GRANT AV,-122.405254,37.789253,"(37.7892525040522, -122.405254463024)",15004881706363
3,3,150380578,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,Friday,05/01/2015,16:44,TARAVAL,"ARREST, BOOKED",1900 Block of 19TH AV,-122.476096,37.751433,"(37.7514328843776, -122.476096065298)",15038057806302
5,5,150380669,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Saturday,05/02/2015,00:42,PARK,"ARREST, BOOKED",DIVISADERO ST / OFARRELL ST,-122.439299,37.782449,"(37.7824492641129, -122.43929911672)",15038066965016
6,6,140009459,ARSON,ARSON,Saturday,01/04/2014,03:52,NORTHERN,"ARREST, BOOKED",SACRAMENTO ST / POLK ST,-122.420875,37.791494,"(37.7914943051906, -122.420874632415)",14000945926030


In [6]:
crimes.shape[0], real_crimes.shape[0]

(1901240, 1698873)

## Make the index

In [7]:
from density_estimation import point_density

In [8]:
bandwidth = 0.01
crime_density = tracts.apply(lambda c: point_density((c['latitude'], c['longitude']), 
                                                     (real_crimes['Y'], real_crimes['X']),
                                                     kernel='gaussian', bw=bandwidth), axis=1)
crime_density *= 100. / crime_density.max()

In [12]:
df = pd.read_csv('features.csv')
df.head()

,tractid,latitude,longitude,crime_index
0,6075010100,37.808420,-122.408579,10.896802
1,6075010200,37.805861,-122.422925,11.077546
2,6075010300,37.801620,-122.415681,21.613337
3,6075010400,37.802273,-122.407558,21.119650
4,6075010500,37.802684,-122.399059,12.413430


In [10]:
df['crime_index'] = crime_density
df.head()

,tractid,latitude,longitude,crime_index
0,6075010100,37.808420,-122.408579,10.896802
1,6075010200,37.805861,-122.422925,11.077546
2,6075010300,37.801620,-122.415681,21.613337
3,6075010400,37.802273,-122.407558,21.119650
4,6075010500,37.802684,-122.399059,12.413430


In [11]:
df.to_csv('features.csv', index=False)